This is a 3D object detection model training & evaluation pipeline based on mmdetection3d from CUHK OpenMMlab.

**citation**:

MMDetection3D Contributors. (2020). OpenMMLab's Next-generation Platform for General 3D Object Detection [Computer software]. https://github.com/open-mmlab/mmdetection3d

# Install mmdetection3d

In [1]:
# check the torch version in colab
!pip list | grep torch

torch                         2.0.1+cu118
torchaudio                    2.0.2+cu118
torchdata                     0.6.1
torchsummary                  1.5.1
torchtext                     0.15.2
torchvision                   0.15.2+cu118


In [ ]:
# install MMEngine, MMCV and MMDetection using MIM
!pip3 install openmim
!mim install mmengine
!mim install "mmcv>=2.0.0rc4,<2.1.0"
!mim install "mmdet>=3.0.0,<3.1.0"

In [ ]:
# install MMdetection3D from source
!git clone https://github.com/open-mmlab/mmdetection3d.git -b dev-1.x
%cd mmdetection3d
!pip install -e .

In [1]:
# verification
import mmdet3d
print(mmdet3d.__version__)

1.1.1


# Download KITTI DataSet


In [2]:
# left color image
!wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip
# velodyne point cloud
!wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_velodyne.zip
# camera calibration matrices
!wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_calib.zip
# labels
!wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip

--2023-06-15 01:42:56--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.170.101
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.170.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12569945557 (12G) [application/zip]
Saving to: ‘data_object_image_2.zip’

data_object_image_2 100%[===================>]  11.71G  25.5MB/s    in 7m 48s  

2023-06-15 01:50:44 (25.6 MB/s) - ‘data_object_image_2.zip’ saved [12569945557/12569945557]

--2023-06-15 01:50:44--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_velodyne.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.169.197
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.169.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28750710812 (27G) [application/zip]
Saving to: 

In [18]:
import os
path = os.getcwd()

/content


In [14]:
!mkdir ./data/kitti/ && mkdir ./data/kitti/ImageSets

# Download data split
!wget -c  https://raw.githubusercontent.com/traveller59/second.pytorch/master/second/data/ImageSets/test.txt --no-check-certificate --content-disposition -O ./data/kitti/ImageSets/test.txt
!wget -c  https://raw.githubusercontent.com/traveller59/second.pytorch/master/second/data/ImageSets/train.txt --no-check-certificate --content-disposition -O ./data/kitti/ImageSets/train.txt
!wget -c  https://raw.githubusercontent.com/traveller59/second.pytorch/master/second/data/ImageSets/val.txt --no-check-certificate --content-disposition -O ./data/kitti/ImageSets/val.txt
!wget -c  https://raw.githubusercontent.com/traveller59/second.pytorch/master/second/data/ImageSets/trainval.txt --no-check-certificate --content-disposition -O ./data/kitti/ImageSets/trainval.txt

mkdir: cannot create directory ‘./data/kitti/’: No such file or directory
./data/kitti/ImageSets/test.txt: No such file or directory
./data/kitti/ImageSets/train.txt: No such file or directory
./data/kitti/ImageSets/val.txt: No such file or directory
./data/kitti/ImageSets/trainval.txt: No such file or directory


In [3]:
# unzip the images to each folder
!mv

sh: 0: can't access tty; job control turned off
sh: 9: shopt: not found
sh: 17: shopt: not found
\[\033\]\w\[\033\]$ 
\[\033\]\w\[\033\]$ 
\[\033\]\w\[\033\]$ 
\[\033\]\w\[\033\]$ ^C


In [ ]:
!python tools/create_data.py kitti --root-path ./data/kitti --out-dir ./data/kitti --extra-tag kitti

# Official Demo

In [1]:
from mmdet3d.apis import inference_detector, init_model
from mmdet3d.registry import VISUALIZERS
from mmdet3d.utils import register_all_modules

In [2]:
# register all modules in mmdet3d into the registries
register_all_modules()

In [12]:
config_file = '../configs/3dssd/3dssd_4xb4_kitti-3d-car.py'
# download the checkpoint from model zoo and put it in `checkpoints/`
!mkdir work_dirs
checkpoint_file = '../work_dirs/second/epoch_40.pth'

In [ ]:
# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cuda:0')

In [ ]:
# init visualizer
visualizer = VISUALIZERS.build(model.cfg.visualizer)
visualizer.dataset_meta = {
    'CLASSES': model.CLASSES,
    'PALETTE': model.PALETTE
}

In [ ]:
# test a single sample
pcd = './data/kitti/000008.bin'
result, data = inference_detector(model, pcd)
points = data['inputs']['points']
data_input = dict(points=points)

In [ ]:
# show the results
out_dir = './'
visualizer.add_datasample(
    'result',
    data_input,
    data_sample=result,
    draw_gt=False,
    show=True,
    wait_time=0,
    out_file=out_dir,
    vis_task='det')